# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4870362e80>
├── 'a' --> tensor([[-0.4269,  0.2989,  0.8096],
│                   [-1.3658,  0.0782,  1.6332]])
└── 'x' --> <FastTreeValue 0x7f4870362f10>
    └── 'c' --> tensor([[ 0.3432, -2.4007,  0.4068,  0.5582],
                        [-0.0710,  1.1531,  0.6361,  1.8184],
                        [-0.6108, -0.1544, -0.0995, -0.4696]])

In [4]:
t.a

tensor([[-0.4269,  0.2989,  0.8096],
        [-1.3658,  0.0782,  1.6332]])

In [5]:
%timeit t.a

74.1 ns ± 0.381 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4870362e80>
├── 'a' --> tensor([[-1.8748, -1.5720,  0.0064],
│                   [-1.0775, -1.4039, -0.5563]])
└── 'x' --> <FastTreeValue 0x7f4870362f10>
    └── 'c' --> tensor([[ 0.3432, -2.4007,  0.4068,  0.5582],
                        [-0.0710,  1.1531,  0.6361,  1.8184],
                        [-0.6108, -0.1544, -0.0995, -0.4696]])

In [7]:
%timeit t.a = new_value

82.9 ns ± 0.757 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.4269,  0.2989,  0.8096],
               [-1.3658,  0.0782,  1.6332]]),
    x: Batch(
           c: tensor([[ 0.3432, -2.4007,  0.4068,  0.5582],
                      [-0.0710,  1.1531,  0.6361,  1.8184],
                      [-0.6108, -0.1544, -0.0995, -0.4696]]),
       ),
)

In [10]:
b.a

tensor([[-0.4269,  0.2989,  0.8096],
        [-1.3658,  0.0782,  1.6332]])

In [11]:
%timeit b.a

71 ns ± 0.552 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.3272,  1.3980, -0.3605],
               [ 0.8914, -0.6359, -1.2176]]),
    x: Batch(
           c: tensor([[ 0.3432, -2.4007,  0.4068,  0.5582],
                      [-0.0710,  1.1531,  0.6361,  1.8184],
                      [-0.6108, -0.1544, -0.0995, -0.4696]]),
       ),
)

In [13]:
%timeit b.a = new_value

570 ns ± 1.56 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

980 ns ± 10.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.8 µs ± 88.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

163 µs ± 780 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

162 µs ± 1.12 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f47c3002b80>
├── 'a' --> tensor([[[-0.4269,  0.2989,  0.8096],
│                    [-1.3658,  0.0782,  1.6332]],
│           
│                   [[-0.4269,  0.2989,  0.8096],
│                    [-1.3658,  0.0782,  1.6332]],
│           
│                   [[-0.4269,  0.2989,  0.8096],
│                    [-1.3658,  0.0782,  1.6332]],
│           
│                   [[-0.4269,  0.2989,  0.8096],
│                    [-1.3658,  0.0782,  1.6332]],
│           
│                   [[-0.4269,  0.2989,  0.8096],
│                    [-1.3658,  0.0782,  1.6332]],
│           
│                   [[-0.4269,  0.2989,  0.8096],
│                    [-1.3658,  0.0782,  1.6332]],
│           
│                   [[-0.4269,  0.2989,  0.8096],
│                    [-1.3658,  0.0782,  1.6332]],
│           
│                   [[-0.4269,  0.2989,  0.8096],
│                    [-1.3658,  0.0782,  1.6332]]])
└── 'x' --> <FastTreeValue 0x7f47c30f3250>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

38.1 µs ± 139 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f48703baf70>
├── 'a' --> tensor([[-0.4269,  0.2989,  0.8096],
│                   [-1.3658,  0.0782,  1.6332],
│                   [-0.4269,  0.2989,  0.8096],
│                   [-1.3658,  0.0782,  1.6332],
│                   [-0.4269,  0.2989,  0.8096],
│                   [-1.3658,  0.0782,  1.6332],
│                   [-0.4269,  0.2989,  0.8096],
│                   [-1.3658,  0.0782,  1.6332],
│                   [-0.4269,  0.2989,  0.8096],
│                   [-1.3658,  0.0782,  1.6332],
│                   [-0.4269,  0.2989,  0.8096],
│                   [-1.3658,  0.0782,  1.6332],
│                   [-0.4269,  0.2989,  0.8096],
│                   [-1.3658,  0.0782,  1.6332],
│                   [-0.4269,  0.2989,  0.8096],
│                   [-1.3658,  0.0782,  1.6332]])
└── 'x' --> <FastTreeValue 0x7f47c3720af0>
    └── 'c' --> tensor([[ 0.3432, -2.4007,  0.4068,  0.5582],
                        [-0.0710,  1.1531,  0.6361,  1.8184],
                 

In [23]:
%timeit t_cat(trees)

35.4 µs ± 198 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

65 µs ± 378 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.4269,  0.2989,  0.8096],
                [-1.3658,  0.0782,  1.6332]],
       
               [[-0.4269,  0.2989,  0.8096],
                [-1.3658,  0.0782,  1.6332]],
       
               [[-0.4269,  0.2989,  0.8096],
                [-1.3658,  0.0782,  1.6332]],
       
               [[-0.4269,  0.2989,  0.8096],
                [-1.3658,  0.0782,  1.6332]],
       
               [[-0.4269,  0.2989,  0.8096],
                [-1.3658,  0.0782,  1.6332]],
       
               [[-0.4269,  0.2989,  0.8096],
                [-1.3658,  0.0782,  1.6332]],
       
               [[-0.4269,  0.2989,  0.8096],
                [-1.3658,  0.0782,  1.6332]],
       
               [[-0.4269,  0.2989,  0.8096],
                [-1.3658,  0.0782,  1.6332]]]),
    x: Batch(
           c: tensor([[[ 0.3432, -2.4007,  0.4068,  0.5582],
                       [-0.0710,  1.1531,  0.6361,  1.8184],
                       [-0.6108, -0.1544, -0.0995, -0.4696]],
         

In [26]:
%timeit Batch.stack(batches)

92.8 µs ± 1.56 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.4269,  0.2989,  0.8096],
               [-1.3658,  0.0782,  1.6332],
               [-0.4269,  0.2989,  0.8096],
               [-1.3658,  0.0782,  1.6332],
               [-0.4269,  0.2989,  0.8096],
               [-1.3658,  0.0782,  1.6332],
               [-0.4269,  0.2989,  0.8096],
               [-1.3658,  0.0782,  1.6332],
               [-0.4269,  0.2989,  0.8096],
               [-1.3658,  0.0782,  1.6332],
               [-0.4269,  0.2989,  0.8096],
               [-1.3658,  0.0782,  1.6332],
               [-0.4269,  0.2989,  0.8096],
               [-1.3658,  0.0782,  1.6332],
               [-0.4269,  0.2989,  0.8096],
               [-1.3658,  0.0782,  1.6332]]),
    x: Batch(
           c: tensor([[ 0.3432, -2.4007,  0.4068,  0.5582],
                      [-0.0710,  1.1531,  0.6361,  1.8184],
                      [-0.6108, -0.1544, -0.0995, -0.4696],
                      [ 0.3432, -2.4007,  0.4068,  0.5582],
                      [-0.0710,  

In [28]:
%timeit Batch.cat(batches)

162 µs ± 1.06 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

372 µs ± 9.14 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
